# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from plotly.subplots import make_subplots
from pytorch_tabular import TabularModel
import torch
import plotly.graph_objects as go
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.impute import KNNImputer
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
import datetime
from collections import Counter
from matplotlib.ticker import MaxNLocator
from itertools import chain
from sklearn.metrics import mean_absolute_error
import pyaging as pya
import matplotlib.lines as mlines
from src.models.simage.tabular.widedeep.ft_transformer import WDFTTransformerModel
import statsmodels.formula.api as smf
from itertools import chain
from pingouin import ancova
from sklearn.preprocessing import LabelEncoder 
import upsetplot

# Load data and model

In [ ]:
path_save = f"E:/YandexDisk/Work/bbd/simage/01_local_explainability"
path_load = f"E:/YandexDisk/Work/pydnameth/datasets/GPL21145/GSEUNN/data/immuno/models/SImAge"

model = WDFTTransformerModel.load_from_checkpoint(checkpoint_path=f"{path_load}/best_fold_0002.ckpt")
model.eval()
model.freeze()
model.to('cpu')

data = pd.read_excel(f"{path_load}/data.xlsx", index_col=0)
data_ctrl = data[data['Status'] == 'Control']
feats = pd.read_excel(f"{path_load}/feats_con_top10.xlsx", index_col=0).index.values

def predict_func(x):
    batch = {
        'all': torch.from_numpy(np.float32(x)),
        'continuous': torch.from_numpy(np.float32(x)),
        'categorical': torch.from_numpy(np.int32(x[:, []])),
    }
    return model(batch).cpu().detach().numpy()

# Local explainability

In [ ]:
samples = {
    'control_negative_aa': 'trn_val_142',
    'control_positive_aa': 'tst_ctrl_004',
    'control_zero_aa': 'trn_val_254',
    'esrd_positive_aa': 'tst_esrd_002'
}

for sample_type, sample_id in samples.items():

    trgt_age = data.at[sample_id, 'Age']
    trgt_pred = data.at[sample_id, 'SImAge']
    trgt_aa = trgt_pred - trgt_age
    print(trgt_age)
    print(trgt_pred)

    n_closest = 32
    data_closest = data_ctrl.iloc[(data_ctrl['SImAge'] - trgt_age).abs().argsort()[:n_closest]]

    explainer = shap.SamplingExplainer(predict_func, data_closest.loc[:, feats].values)
    print(explainer.expected_value)
    shap_values = explainer.shap_values(data.loc[[sample_id], feats].values)[0]
    shap_values = shap_values * (trgt_pred - trgt_age) / (trgt_pred - explainer.expected_value)

    shap.plots.waterfall(
        shap.Explanation(
            values=shap_values,
            base_values=trgt_age,
            data=data.loc[sample_id, feats].values,
            feature_names=feats
        ),
        max_display=len(feats),
        show=False,
    )
    fig = plt.gcf()
    fig.savefig(f"{path_save}/{sample_type}.pdf", bbox_inches='tight')
    fig.savefig(f"{path_save}/{sample_type}.png", bbox_inches='tight')
    plt.close()